<a href="https://colab.research.google.com/github/sreramk1/sentiment-analysis/blob/main/training_notebooks/train_sentiment_p1v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training the Model for the first time (initialized with random weights)

In this notebook, I had used the classes I had written to simplify the training process for the sentiment analysis model: https://github.com/sreramk1/sentiment-analysis.git

In [1]:
!rm -r sentiment-analysis
!git clone https://github.com/sreramk1/sentiment-analysis.git

Cloning into 'sentiment-analysis'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (156/156), done.
remote: Total 225 (delta 120), reused 146 (delta 60), pack-reused 0
Receiving objects: 100% (225/225), 15.18 MiB | 4.59 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [2]:
import os
import sys

sys.path.append('/content/sentiment-analysis')
os.environ['PYTHONPATH'] += ":/content/sentiment-analysis"

In [3]:
from util.jupyter_helpers import display_tf_ds

from train_validate_predict.pandas_read_ds_from_csv import PandasReadDatasetFromCSV
from sentiment_v1.preprocess_ds import TweetReviewCsvToPdDataFrame
from sentiment_v1.convert_ds_to_tensors import TweetReviewTfDataset
from sentiment_v1.model_builder import SentimentLSTMDense64Dense32Dense16Dense8Dense1
from train_validate_predict.model_train_validate_predict import ModelTrainValidatePredict
from model.keras_sequence_model_weights import KerasSequenceModelWeights

In [4]:
pd_dataset = PandasReadDatasetFromCSV("/content/sentiment-analysis/third_party/dataset/airline_sentiment_analysis.csv")
pd_dataset.load_dataset()
tweet_review_csv_to_pd_dataframe = TweetReviewCsvToPdDataFrame(pd_dataset)
tweet_review_tf_ds = TweetReviewTfDataset(tweet_review_csv_to_pd_dataframe)

In [5]:
model_builder = SentimentLSTMDense64Dense32Dense16Dense8Dense1(tf_ds_store=tweet_review_tf_ds)
model_builder.build_model()
model = model_builder.get_model()
model_trainer = ModelTrainValidatePredict(tweet_review_tf_ds, model)
model_trainer.predict("This is a bad airline")

array([[-0.00026961]], dtype=float32)

In [6]:
model_trainer.train()
model_trainer.evaluate()

Epoch 1/10
145/145 [==============================] - 20s 90ms/step - loss: 0.6023 - accuracy: 0.7953 - val_loss: 0.4349 - val_accuracy: 0.7951
Epoch 2/10
145/145 [==============================] - 10s 67ms/step - loss: 0.4013 - accuracy: 0.7953
Epoch 3/10
145/145 [==============================] - 10s 67ms/step - loss: 0.2834 - accuracy: 0.8458
Epoch 4/10
145/145 [==============================] - 10s 67ms/step - loss: 0.1769 - accuracy: 0.9389
Epoch 5/10
145/145 [==============================] - 10s 67ms/step - loss: 0.1289 - accuracy: 0.9572
Epoch 6/10
145/145 [==============================] - 10s 67ms/step - loss: 0.0987 - accuracy: 0.9687
Epoch 7/10
145/145 [==============================] - 10s 67ms/step - loss: 0.0767 - accuracy: 0.9766
Epoch 8/10
145/145 [==============================] - 10s 67ms/step - loss: 0.0605 - accuracy: 0.9820
Epoch 9/10
145/145 [==============================] - 10s 67ms/step - loss: 0.0471 - accuracy: 0.9866
Epoch 10/10
3/3 [=======================

(0.31747668981552124, 0.9129493236541748)

In [7]:
print("after training, bad review:" + str(model_trainer.predict("This is a bad airline")))
print("after training, good review:" + str(model_trainer.predict("This is a good airline")))

after training, bad review:[[-1.3539096]]
after training, good review:[[2.8381493]]


# Create a new model, load the trained weights from the file into it, and predict 

In [8]:
sequence_model_weights = KerasSequenceModelWeights()
sequence_model_weights.set_model(model)
sequence_model_weights.read_weights_from_model()
sequence_model_weights.write_weights_to_file("weights.json")

In [9]:
another_sequence_model_weights = KerasSequenceModelWeights()
loaded_weights = another_sequence_model_weights.load_weights_from_file("weights.json")

second_model_builder = SentimentLSTMDense64Dense32Dense16Dense8Dense1(layered_weights=loaded_weights)
second_model_builder.build_model()
second_model = second_model_builder.get_model()
second_model_tvp = ModelTrainValidatePredict(tweet_review_tf_ds, second_model)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [10]:
another_sequence_model_weights.set_model(second_model)
another_sequence_model_weights.set_weights_to_model()

print("------------------------------------------------------------")
print("for a bad review" + str(second_model_tvp.predict("This is a bad airline")))
print("for a good review" + str(second_model_tvp.predict("This is a good airline")))
print("------------------------------------------------------------")

------------------------------------------------------------
for a bad review[[-1.3539096]]
for a good review[[2.8381493]]
------------------------------------------------------------
